<a href="https://colab.research.google.com/github/nathelf/LLM_Mestrado/blob/main/valorant_doc_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# ✅ Instalando pacotes necessários
!pip install -q transformers accelerate markdown2 python-docx


In [4]:

# ✅ Importação das bibliotecas essenciais
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import os
import json
from docx import Document
from google.colab import files
from zipfile import ZipFile
import shutil
import gc
from datetime import datetime


In [5]:

# ✅ Autodetecção de CPU ou GPU
use_gpu = torch.cuda.is_available()
device_id = 0 if use_gpu else -1

# ✅ Modelo pequeno e eficiente
model_id = "google/flan-t5-base"

# ✅ Carrega tokenizer uma vez
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:

# ✅ Função que carrega, gera a documentação e libera o modelo
def gerar_documentacao(prompt, max_tokens=512):
    """
    Gera documentação a partir do prompt, carregando e liberando o modelo a cada execução.
    """
    print("✅ Carregando modelo...")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    pipe = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        device=device_id,
        max_new_tokens=max_tokens
    )

    print("✅ Gerando documentação...")
    output = pipe(prompt)[0]['generated_text']

    print("✅ Liberando modelo...")
    del model, pipe
    gc.collect()
    if use_gpu:
        torch.cuda.empty_cache()

    return output



In [7]:

# ✅ Prompt padrão para gerar a documentação técnica
prompt_base = '''Sua tarefa é analisar um objeto JSON contendo dados de um agente da API pública do jogo Valorant e gerar uma documentação técnica clara, precisa e adequada para desenvolvedores.

---

1. **Título:** Nome do agente (campo `displayName`)
2. **Descrição geral** explicando o papel e o uso do agente no jogo.
3. **Tabela com os principais campos**:
   | Campo | Tipo de dado | Descrição |
   |-------|--------------|-----------|
   Preencha com os campos principais do JSON.
4. **Subseções** para objetos aninhados como `role`, `abilities` e outros.
5. Use Markdown estruturado (títulos `##`, listas `-` e tabelas).
6. Não repita o JSON nem inclua comentários irrelevantes.

JSON de entrada:

{{json}}

Gere a documentação de forma clara, objetiva e técnica.
'''


In [8]:

# ✅ Criando diretórios
os.makedirs("json_valorant", exist_ok=True)
os.makedirs("resultados", exist_ok=True)
os.makedirs("docx", exist_ok=True)

# ✅ Upload dos arquivos
print("🔼 Envie os arquivos .json dos agentes (um por arquivo)...")
uploaded = files.upload()

# ✅ Movendo arquivos enviados para a pasta
for nome in uploaded:
    shutil.move(nome, f"json_valorant/{nome}")


🔼 Envie os arquivos .json dos agentes (um por arquivo)...


Saving astra.json to astra.json


In [17]:

# ✅ Função para converter Markdown em estrutura de DOCX
def parse_markdown_to_docx(md_text: str, doc: Document):
    tabela = None
    for linha in md_text.split("\n"):
        linha = linha.strip()
        if linha.startswith("##"):
            doc.add_heading(linha.replace("##", "").strip(), level=2)
            tabela = None
        elif linha.startswith("#"):
            doc.add_heading(linha.replace("#", "").strip(), level=3)
            tabela = None
        elif linha.startswith("|") and "|" in linha[1:]:
            if "Campo" in linha or "---" in linha:
                continue
            valores = [c.strip() for c in linha.split("|") if c.strip()]
            if tabela is None:
                tabela = doc.add_table(rows=1, cols=len(valores))
                tabela.style = 'Table Grid'
            row = tabela.add_row()
            for i, val in enumerate(valores):
                row.cells[i].text = val
        elif linha.startswith("-") or linha.startswith("*"):
            doc.add_paragraph(linha[1:].strip(), style="List Bullet")
        elif linha:
            doc.add_paragraph(linha)


In [19]:
# ✅ Função auxiliar para "achatar" JSONs aninhados em um único dicionário
def flatten_json(y, prefix=''):
    out = {}
    for k, v in y.items():
        if isinstance(v, dict):
            out.update(flatten_json(v, prefix + k + '.'))
        elif isinstance(v, list):
            out[prefix + k] = str(v)
        else:
            out[prefix + k] = v
    return out


In [20]:
# ✅ Processamento de cada JSON e geração de tabela
for filename in os.listdir("json_valorant"):
    if not filename.endswith(".json"):
        continue

    with open(f"json_valorant/{filename}", "r", encoding="utf-8") as f:
        data = json.load(f)

    prompt = prompt_base.replace("{{json}}", json.dumps(data, indent=2, ensure_ascii=False))

    print(f"📄 Gerando documentação para: {filename}")
    output = gerar_documentacao(prompt, max_tokens=3000)

    # ✅ Salvando Markdown (opcional)
    md_path = f"resultados/{filename.replace('.json', '.md')}"
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(output)

    # ✅ Gerando DOCX
    doc = Document()
    doc.add_heading(f"Documentação Técnica – {data.get('displayName', filename)}", level=1)
    doc.add_paragraph("Projeto: API Valorant", style="Intense Quote")
    doc.add_paragraph("Data: " + datetime.now().strftime("%d/%m/%Y"), style="Intense Quote")
    doc.add_paragraph("\n")

    # ✅ Achata o JSON para representar tudo como tabela
    flat_data = flatten_json(data)

    # ✅ Cria tabela com todas as informações
    table = doc.add_table(rows=1, cols=2)
    table.style = 'Table Grid'
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Campo'
    hdr_cells[1].text = 'Valor'

    for k, v in flat_data.items():
        row_cells = table.add_row().cells
        row_cells[0].text = str(k)
        row_cells[1].text = str(v)

    docx_path = f"docx/{filename.replace('.json', '.docx')}"
    doc.save(docx_path)

    print(f"✅ Documentação gerada para {filename}")


📄 Gerando documentação para: astra.json
✅ Carregando modelo...


Device set to use cpu


✅ Gerando documentação...
✅ Liberando modelo...
✅ Documentação gerada para astra.json


In [21]:

# ✅ Compactando os arquivos .docx
with ZipFile("documentacoes_valorant.zip", "w") as zipf:
    for file in os.listdir("docx"):
        zipf.write(f"docx/{file}", arcname=file)

# ✅ Download do zip
files.download("documentacoes_valorant.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>